# Imports

In [1]:
import joblib

import constants.paths as pth
from src.preprocessing import preprocess_data  # noqa: F401
from src.utils.load_data import load_data


In [ ]:
data = load_data()
data.columns
if ["signal_1", "signal_3"] 

Index(['sex', 'employment', 'married', 'income', 'signal1', 'signal2',
       'signal3', 'default'],
      dtype='object')

In [2]:
data_to_predict = load_data(pth.NEW_LOANS_PATH)
scaled_data_to_predict = preprocess_data(data_to_predict)

KeyError: "['signal1', 'signal3'] not in index"

In [ ]:
# Make predictions
model_name = "hist_gradient_boosting"  # Change as needed
model = joblib.load(pth.MODELS_DIR / f"{model_name}.pkl")  # Load your trained model

proba_predictions = model.predict_proba(scaled_data_to_predict)[
    :, 1
]  # Probability of the positive class (default probability)

In [ ]:
data_to_predict["predicted_default_proba"] = proba_predictions

In [ ]:
# We compute the breake-even interest rate for each loan
data_to_predict["breakeven_interest_rate"] = data_to_predict[
    "predicted_default_proba"
] / (1 - data_to_predict["predicted_default_proba"])

In [ ]:
data_to_predict[data_to_predict["predicted_default_proba"] > 0.5]

,id,sex,employment,married,income,signal2,predicted_default_proba,breakeven_interest_rate
1,2,F,retired,1,8.722580,0.000000,0.524108,1.101316
3,4,M,employed,1,9.039433,0.624570,0.529336,1.124658
4,5,M,retired,1,7.944492,0.097230,0.731141,2.719428
5,6,M,student,0,-9999.000000,0.918534,0.565818,1.303182
7,8,M,unemployed,1,-9999.000000,0.000000,0.757001,3.115243
...,...,...,...,...,...,...,...,...
99987,99988,M,employed,0,8.827028,0.621411,0.598913,1.493225
99988,99989,M,employed,1,9.250714,0.196476,0.593072,1.457438
99989,99990,M,employed,1,9.152287,0.000000,0.512587,1.051647
99996,99997,M,student,0,8.736971,0.635176,0.522740,1.095294


The function
$$
f : x \mapsto \frac{x}{1-x}
$$
is strictly growing on $[0,1[$ and $f(\frac{1}{2})=1$.

Thus, any predicted default probability above $0.5$ gives a break even interest rate above $100\%$. We cannot fix interest rates above $100\%$.

In [ ]:
data_to_predict[data_to_predict["breakeven_interest_rate"] < 0.1]

,id,sex,employment,married,income,signal2,predicted_default_proba,breakeven_interest_rate
8,9,F,unemployed,0,10.170878,0.719767,0.057620,0.061143
12,13,M,employed,1,10.187914,0.787726,0.077207,0.083667
14,15,F,employed,0,10.272773,0.846993,0.046394,0.048651
20,21,M,retired,0,10.886951,0.342223,0.040472,0.042179
49,50,M,employed,1,10.275465,0.744132,0.080368,0.087392
...,...,...,...,...,...,...,...,...
99979,99980,M,employed,1,10.684577,0.660603,0.059851,0.063661
99986,99987,F,student,0,10.521965,0.738878,0.037005,0.038427
99993,99994,F,employed,1,10.947802,0.587365,0.009997,0.010098
99994,99995,M,student,0,11.155950,0.000000,0.024894,0.025529


In [ ]:
threshold = 0.6214760622417385
predictions = (proba_predictions >= threshold).astype(int)

In [ ]:
predictions.sum()

np.int64(21927)

In [ ]:
data_to_predict["predicted_default_proba"].hist(bins=50)